# \[Ignore\] Learning by Examples

## Simple SDP solver

In [6]:
import cvxpy as cp
import numpy as np

# Generate a random SDP.
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.

constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]

prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(solver=cp.CVXOPT,verbose=True)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.5678e+00  1.5678e+00  9e+00  3e+00  4e-01  1e+00
 1:  2.4030e+00  2.4911e+00  5e-01  3e-01  4e-02  2e-01
 2:  2.6535e+00  2.6672e+00  7e-02  4e-02  5e-03  3e-02
 3:  2.6456e+00  2.6514e+00  2e-02  1e-02  2e-03  1e-02
 4:  2.6544e+00  2.6555e+00  5e-03  3e-03  4e-04  2e-03
 5:  2.6540e+00  2.6542e+00  1e-03  7e-04  1e-04  5e-04
 6:  2.6544e+00  2.6544e+00  3e-04  2e-04  2e-05  1e-04
 7:  2.6543e+00  2.6543e+00  7e-05  4e-05  6e-06  3e-05
 8:  2.6543e+00  2.6544e+00  2e-05  9e-06  1e-06  6e-06
 9:  2.6543e+00  2.6543e+00  3e-06  2e-06  2e-07  1e-06
10:  2.6543e+00  2.6543e+00  6e-07  3e-07  5e-08  2e-07
11:  2.6543e+00  2.6543e+00  1e-07  5e-08  7e-09  3e-08
Optimal solution found.
The optimal value is 2.6543479996361037
A solution X is
[[ 1.60816352 -0.59785914 -0.69585439]
 [-0.59785914  0.22238688  0.24705027]
 [-0.69585439  0.24705027  1.3970322 ]]


# \[Ignore\] Figuring how to use Tensor products

In [18]:
from sympy.physics.quantum.qubit import Qubit
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import TensorProduct
import sympy.physics.quantum as sq
import sympy as sp

## Extending the functionality of sympy Quantum 
(imported from the swapKCBS project)

### Extending tensor

In [23]:

def powerDrop(expr):
    if isinstance(expr,sp.Pow): #TODO: make sure the base is not too complex
        # print("PowerEncountered")
        if expr.exp>=2:
            # print("glaba")
            # display(expr.base)
            _=sq.qapply(sp.Mul(expr.base,expr.base))
            if expr.exp>2:
                return powerDrop(_*sp.Pow(expr.base,expr.exp-2))
            else:
                return _
        else:
            return expr #autoDropDim(sp.Mul(expr.base,expr.base))
    else:
        if expr.has(sp.Pow):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add) or isinstance(expr,sq.TensorProduct):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(powerDrop(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                elif isinstance(expr,sq.TensorProduct):
                    return sq.TensorProduct(*new_args)  

            else:
                return expr
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        
    
def autoDropDim(expr):
    #print("Expression")
    #if isinstance(expr,sp.Mul):
        #print("type:multiplier")
    #display(expr)
    
    
    if isinstance(expr,sq.TensorProduct):
        new_args=[]
        for _ in expr.args:
            #display(_)
            #print(type(_))
            if _ != sp.Integer(1):
            #if not isinstance(_,core.numbers.One):
                new_args.append(_)
        #print("TensorProduct with %d non-ones in the tensor product"%len(new_args))
        if(len(new_args)==0):
            return sp.Integer(1)
        else:
            return sq.TensorProduct(*new_args)
    else:
        if expr.has(sq.TensorProduct):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(autoDropDim(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr #when the expression is just an integer or some such


        
def tsimp(e,pruneMe=True):
    res=sq.qapply(powerDrop(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit()))
    if pruneMe:
        return prune(res)
    else:
        return res

def tdsimp(e,pruneMe=True):
    res=autoDropDim(sq.qapply(powerDrop(autoDropDim(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit()))))
    if pruneMe:
        return prune(res)
    else:
        return res
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(e).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(sq.qapply(e)).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp(e).doit())

### Prune

In [24]:
def prune(expr,thr=10,remNum=False):
    if isinstance(expr,sp.Number): 
        if remNum==False:
            if sp.Abs(expr)<10**(-thr):
                return sp.Integer(0)
            else:
                return expr
        else:
            return sp.Integer(1)
    else:
        if expr.has(sp.Number):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add) or isinstance(expr,sq.TensorProduct):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(prune(_,thr,remNum))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                elif isinstance(expr,sq.TensorProduct):
                    return sq.TensorProduct(*new_args)  

            else:
                return expr
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        

# test=(A[0]*2)
# test.has(sp.Number)
# prune(test,remNum=True)

## Testing now

In [25]:
k0=Qubit(0)
k1=Qubit(1)

In [35]:
v=TensorProduct(k0,k1)
w=TensorProduct(k1,k1)
#sp.Integer(1)

In [36]:
w

|1>x|1>

In [30]:
tdsimp(Dagger(v)*v)

1

In [31]:
tdsimp(Dagger(w)*v)

0

# Libraries

In [5]:
from sympy.physics.quantum.qubit import Qubit
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import TensorProduct as TP
import sympy.physics.quantum as sq
import sympy as sp


import cvxpy as cp
import numpy as np
import itertools

## Extending the tensor product functionality

### Prune

In [6]:
def prune(expr,thr=10,remNum=False):
    if isinstance(expr,sp.Number): 
        if remNum==False:
            if sp.Abs(expr)<10**(-thr):
                return sp.Integer(0)
            else:
                return expr
        else:
            return sp.Integer(1)
    else:
        if expr.has(sp.Number):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add) or isinstance(expr,sq.TensorProduct):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(prune(_,thr,remNum))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                elif isinstance(expr,sq.TensorProduct):
                    return sq.TensorProduct(*new_args)  

            else:
                return expr
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        

# test=(A[0]*2)
# test.has(sp.Number)
# prune(test,remNum=True)

### Extending Tensor

In [7]:

def powerDrop(expr):
    if isinstance(expr,sp.Pow): #TODO: make sure the base is not too complex
        # print("PowerEncountered")
        if expr.exp>=2:
            # print("glaba")
            # display(expr.base)
            _=sq.qapply(sp.Mul(expr.base,expr.base))
            if expr.exp>2:
                return powerDrop(_*sp.Pow(expr.base,expr.exp-2))
            else:
                return _
        else:
            return expr #autoDropDim(sp.Mul(expr.base,expr.base))
    else:
        if expr.has(sp.Pow):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add) or isinstance(expr,sq.TensorProduct):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(powerDrop(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                elif isinstance(expr,sq.TensorProduct):
                    return sq.TensorProduct(*new_args)  

            else:
                return expr
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        
    
def autoDropDim(expr):
    #print("Expression")
    #if isinstance(expr,sp.Mul):
        #print("type:multiplier")
    #display(expr)
    
    
    if isinstance(expr,sq.TensorProduct):
        new_args=[]
        for _ in expr.args:
            #display(_)
            #print(type(_))
            if _ != sp.Integer(1):
            #if not isinstance(_,core.numbers.One):
                new_args.append(_)
        #print("TensorProduct with %d non-ones in the tensor product"%len(new_args))
        if(len(new_args)==0):
            return sp.Integer(1)
        else:
            return sq.TensorProduct(*new_args)
    else:
        if expr.has(sq.TensorProduct):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(autoDropDim(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr #when the expression is just an integer or some such


        
def tsimp(e,pruneMe=True):
    res=sq.qapply(powerDrop(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit()))
    if pruneMe:
        return prune(res)
    else:
        return res

def tdsimp(e,pruneMe=True):
    res=autoDropDim(sq.qapply(powerDrop(autoDropDim(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit()))))
    if pruneMe:
        return prune(res)
    else:
        return res
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(e).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(sq.qapply(e)).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp(e).doit())

## Convert a matrix from Sympy Quantum to Numpy

In [8]:
def rep(ρ):
    return np.array(sq.represent(ρ)).astype(np.complex64)

# Testing, stage 1

## Define the problem, symbolically

In [30]:
k0=Qubit(0)
k1=Qubit(1)

ρ=k0*Dagger(k0)

ρ_=rep(ρ)

In [37]:
print(type(ρ_))
display(ρ)
display(sq.represent(ρ))
display(ρ_)

<class 'numpy.ndarray'>


|0><0|

Matrix([
[1, 0],
[0, 0]])

array([[1., 0.],
       [0., 0.]])

## Define the SDP and solve it

In [36]:
#size of the problem
n=2 #cp.Integer(2)
#ρ_matr=

#density matrix to optimize over
ρ1 = cp.Variable((n,n), symmetric=True)

#density matrix, so positive
constraints = [ρ1 >> 0]
constraints += [cp.trace(ρ1) == 1]

prob = cp.Problem(cp.Maximize(cp.trace(ρ_ @ ρ1)), constraints)

prob.solve(solver=cp.CVXOPT,verbose=True)

                  
#cp.trace(ρ_mat @ ρ1)),                  

     pcost       dcost       gap    pres   dres   k/t
 0: -5.0000e-01 -5.0000e-01  1e+00  2e-16  2e+00  1e+00
 1: -7.6888e-01 -7.7110e-01  2e-01  2e-16  3e-01  2e-01
 2: -9.9769e-01 -9.9802e-01  5e-03  1e-16  7e-03  3e-03
 3: -9.9998e-01 -9.9998e-01  5e-05  2e-16  7e-05  3e-05
 4: -1.0000e+00 -1.0000e+00  5e-07  2e-16  7e-07  3e-07
 5: -1.0000e+00 -1.0000e+00  5e-09  2e-16  7e-09  3e-09
Optimal solution found.


0.9999999976879534

# DI CF with Alice testing rigidity + ET

## Settings

In [9]:
verbose=True

## Bare kets

In [10]:
# I am using an O and an L for denoting 0 and 1 (so visually they are similar and can still be variables in python)
o=Qubit(0)
l=Qubit(1)

I = sp.Integer(1)

D = Dagger

ι = sp.I #ImaginaryUnit

## Initial components of the state

In [11]:

#Convention
# ABC X'Y'Z' Y'' Z'' A' S' S'' R' G' B' C'

ψ_ABC=(TP(o,o,o) + TP(l,l,l))/sp.sqrt(2)
display(ψ_ABC)

ψ_XYZYZ=(TP(o,o,l,o,l) + TP(o,l,o,l,o) + TP(l,o,o,o,o) + TP(l,l,l,l,l))/sp.sqrt(4)
display(ψ_XYZYZ)

#No, I did not do this on purpose
ψ_ASS = TP(o,o,o)
display(ψ_ASS)

#Random number
ψ_R = (o + l )/sp.sqrt(2)
display(ψ_R)

ψ1=TP(ψ_ABC,ψ_XYZYZ,ψ_ASS,ψ_R)
display(tdsimp(ψ1))

sqrt(2)*(|0>x|0>x|0> + |1>x|1>x|1>)/2

(|0>x|0>x|1>x|0>x|1> + |0>x|1>x|0>x|1>x|0> + |1>x|0>x|0>x|0>x|0> + |1>x|1>x|1>x|1>x|1>)/2

|0>x|0>x|0>

sqrt(2)*(|0> + |1>)/2

|0>x|0>x|0>x|0>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|0>/4 + |0>x|0>x|0>x|0>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|1>/4 + |0>x|0>x|0>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|0>x|0>/4 + |0>x|0>x|0>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|0>x|1>/4 + |0>x|0>x|0>x|1>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>/4 + |0>x|0>x|0>x|1>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|1>/4 + |0>x|0>x|0>x|1>x|1>x|1>x|1>x|1>x|0>x|0>x|0>x|0>/4 + |0>x|0>x|0>x|1>x|1>x|1>x|1>x|1>x|0>x|0>x|0>x|1>/4 + |1>x|1>x|1>x|0>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|0>/4 + |1>x|1>x|1>x|0>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|1>/4 + |1>x|1>x|1>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|0>x|0>/4 + |1>x|1>x|1>x|0>x|1>x|0>x|1>x|0>x|0>x|0>x|0>x|1>/4 + |1>x|1>x|1>x|1>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>/4 + |1>x|1>x|1>x|1>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|1>/4 + |1>x|1>x|1>x|1>x|1>x|1>x|1>x|1>x|0>x|0>x|0>x|0>/4 + |1>x|1>x|1>x|1>x|1>x|1>x|1>x|1>x|0>x|0>x|0>x|1>/4

## Unitary

### Pauli X, projectors for measuring in Y and Z

In [12]:
#Convention
# ABC X'Y'Z' Y'' Z'' A' S' S'' R' G' B' C'

Π_z0 = o * D(o)
Π_z1 = l * D(l)
display(Π_z0,Π_z1)

def Π_z(i):
    if i==0:
        return Π_z0
    else:
        return Π_z1

x0 = (o + l)/sp.sqrt(2)
x1 = (o + l)/sp.sqrt(2)

Π_x0 = x0 * D(x0)
Π_x1 = x1 * D(x1)
display(Π_x0,Π_x1)

y0 = (o + ι*l)/sp.sqrt(2)
y1 = (o - ι*l)/sp.sqrt(2)

Π_y0 = y0 * D(y0)
Π_y1 = y1 * D(y1)
display(Π_y0,Π_y1)
#rep(y0)

X_hat = o * D(l) + l *D(o)
display(X_hat)

|0><0|

|1><1|

(|0> + |1>)*(<0| + <1|)/2

(|0> + |1>)*(<0| + <1|)/2

(|0> + I*|1>)*(<0| - I*<1|)/2

(|0> - I*|1>)*(<0| + I*<1|)/2

|0><1| + |1><0|

### Assembling the Unitary and the projector

In [13]:
#Constructing the unitary

#Convention
# ABC X'Y'Z' Y'' Z'' A' S' S'' R' G' B' C'

U_I = TP(Π_x0,I,I,        #ABC
         Π_z0,I,I, I,I,   #XYZ YZ  #ASSRGBC
         I,I,I,I,I,I,I) + \
      TP(Π_x1,I,I,        #ABC
         Π_z0,I,I, I,I,   #XYZ YZ  #ASSRGBC
         X_hat,I,I,I,I,I,I) + \
      TP(Π_y0,I,I,        #ABC
         Π_z1,I,I, I,I,   #XYZ YZ  #ASSRGBC
         I,I,I,I,I,I,I) + \
      TP(Π_y1,I,I,        #ABC
         Π_z1,I,I, I,I,   #XYZ YZ  #ASSRGBC
         X_hat,I,I,I,I,I,I)

#display(U_I)


bi = [0,1]

U_II = None
def f(a,x,r):
    return (a+(x*r))%2

for a,x,r in itertools.product(bi,bi,bi):        
    #print(a,x,r,f(a,x,r))
    if (f(a,x,r)):
        _ = TP(I,I,I,        #ABC
            Π_z(x),I,I, I,I,   #XYZ YZ  
            Π_z(a),X_hat,X_hat,Π_z(r),I,I,I) #ASSRGBC
    else:
        _ = TP(I,I,I,        #ABC
            Π_z(x),I,I, I,I,   #XYZ YZ  
            Π_z(a),I,I,Π_z(r),I,I,I) #ASSRGBC
    
    if U_II==None:
        U_II = _
    else:
        U_II += _
    #Qubit(0)
#display(U_II)


U = U_II * U_I

if(verbose): display(U)
print("Constructed the unitary")

ρ1=U*ψ1*D(U*ψ1)
display(ρ1)



####################Constructing the projector

Π = None
_ = None
#GHZ winning condition
def f(a,b,c,x,y,z,g):
    if( (x+g)%2 == 1 and (a+b+c+1)%2 == (x*y*z)):
        return 1
    else:
        return 0    

#I_ = Π_z0 + Π_z1
    
for a,b,c, x,y,z, g in itertools.product(bi,bi,bi, bi,bi,bi, bi):        
    if (f(a,b,c, x,y,z, g)==1):
        #print(a,b,c, x,y,z, g, f(a,b,c,x,y,z,g))
        _ = TP(I,I,I,        #ABC
            Π_z(x),I,I, Π_z(y),Π_z(z),   #XYZ YZ  
            Π_z(a),I,I,I,Π_z(g),Π_z(b),Π_z(c)) #ASSRGBC
    
        if Π==None:
            Π = _
        else:
            Π += _

if(verbose): 
    display(Π)
    #display(sq.represent(Π))
    #display(rep(Π))
print("Constructed the projector")




(1x1x1x|0><0|x1x1x1x1x|0><0|x1x1x|0><0|x1x1x1 + 1x1x1x|0><0|x1x1x1x1x|0><0|x1x1x|1><1|x1x1x1 + 1x1x1x|0><0|x1x1x1x1x|1><1|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|0><0|x1x1x1 + 1x1x1x|0><0|x1x1x1x1x|1><1|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|1><1|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|0><0|x1x1x|0><0|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|0><0|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|1><1|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|1><1|x1x1x|1><1|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|1><1|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|0><0|x1x1x1)*(((|0> - I*|1>)*(<0| + I*<1|))x1x1x|1><1|x1x1x1x1x(|0><1| + |1><0|)x1x1x1x1x1x1/2 + ((|0> + I*|1>)*(<0| - I*<1|))x1x1x|1><1|x1x1x1x1x1x1x1x1x1x1x1/2 + ((|0> + |1>)*(<0| + <1|))x1x1x|0><0|x1x1x1x1x1x1x1x1x1x1x1/2 + ((|0> + |1>)*(<0| + <1|))x1x1x|0><0|x1x1x1x1x(|0><1| + |1><0|)x1x1x1x1x1x1/2)

Constructed the unitary


(1x1x1x|0><0|x1x1x1x1x|0><0|x1x1x|0><0|x1x1x1 + 1x1x1x|0><0|x1x1x1x1x|0><0|x1x1x|1><1|x1x1x1 + 1x1x1x|0><0|x1x1x1x1x|1><1|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|0><0|x1x1x1 + 1x1x1x|0><0|x1x1x1x1x|1><1|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|1><1|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|0><0|x1x1x|0><0|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|0><0|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|1><1|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|1><1|x1x1x|1><1|x1x1x1 + 1x1x1x|1><1|x1x1x1x1x|1><1|x(|0><1| + |1><0|)x(|0><1| + |1><0|)x|0><0|x1x1x1)*(((|0> - I*|1>)*(<0| + I*<1|))x1x1x|1><1|x1x1x1x1x(|0><1| + |1><0|)x1x1x1x1x1x1/2 + ((|0> + I*|1>)*(<0| - I*<1|))x1x1x|1><1|x1x1x1x1x1x1x1x1x1x1x1/2 + ((|0> + |1>)*(<0| + <1|))x1x1x|0><0|x1x1x1x1x1x1x1x1x1x1x1/2 + ((|0> + |1>)*(<0| + <1|))x1x1x|0><0|x1x1x1x1x(|0><1| + |1><0|)x1x1x1x1x1x1/2)*(|0>x|0>x|0> + |1>x|1>x|1>)x(|0>x|0>x|1>x|0>x|1> + |0>x|1>x|0>x|1>x|0> + |1>x|0>x|0>x|0>x|0> + |1>x|1>x|1>x|1>x|1>)x|0>x|0>x|0>x(|0> + |1>)*(<0|x<0|x<0| + <1|x<1|x<1|)x(<0|x<0|x<1|x<0|x<1| + <0|x<1|x<0|x<

1x1x1x|0><0|x1x1x|0><0|x|0><0|x|0><0|x1x1x1x|1><1|x|0><0|x|1><1| + 1x1x1x|0><0|x1x1x|0><0|x|0><0|x|0><0|x1x1x1x|1><1|x|1><1|x|0><0| + 1x1x1x|0><0|x1x1x|0><0|x|0><0|x|1><1|x1x1x1x|1><1|x|0><0|x|0><0| + 1x1x1x|0><0|x1x1x|0><0|x|0><0|x|1><1|x1x1x1x|1><1|x|1><1|x|1><1| + 1x1x1x|0><0|x1x1x|0><0|x|1><1|x|0><0|x1x1x1x|1><1|x|0><0|x|1><1| + 1x1x1x|0><0|x1x1x|0><0|x|1><1|x|0><0|x1x1x1x|1><1|x|1><1|x|0><0| + 1x1x1x|0><0|x1x1x|0><0|x|1><1|x|1><1|x1x1x1x|1><1|x|0><0|x|0><0| + 1x1x1x|0><0|x1x1x|0><0|x|1><1|x|1><1|x1x1x1x|1><1|x|1><1|x|1><1| + 1x1x1x|0><0|x1x1x|1><1|x|0><0|x|0><0|x1x1x1x|1><1|x|0><0|x|1><1| + 1x1x1x|0><0|x1x1x|1><1|x|0><0|x|0><0|x1x1x1x|1><1|x|1><1|x|0><0| + 1x1x1x|0><0|x1x1x|1><1|x|0><0|x|1><1|x1x1x1x|1><1|x|0><0|x|0><0| + 1x1x1x|0><0|x1x1x|1><1|x|0><0|x|1><1|x1x1x1x|1><1|x|1><1|x|1><1| + 1x1x1x|0><0|x1x1x|1><1|x|1><1|x|0><0|x1x1x1x|1><1|x|0><0|x|1><1| + 1x1x1x|0><0|x1x1x|1><1|x|1><1|x|0><0|x1x1x1x|1><1|x|1><1|x|0><0| + 1x1x1x|0><0|x1x1x|1><1|x|1><1|x|1><1|x1x1x1x|1><1|x|0><0|x|0><

Constructed the projector


In [22]:
a = D(TP(o,o))

In [21]:
allZero=TP(o,o,o,        #ABC
         o,o,o, o,o,   #XYZ YZ  #ASSRGBC
         o,o,o,o,o,o)
display(allZero)
result=Π*D

|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>x|0>

TypeError: as_base_exp() missing 1 required positional argument: 'self'

In [ ]:
#display(U*ψ1)
#ψ1_A=sq.qapply(U*ψ1)
#rep(ψ1)
#rep(U)

## TODO

* sympy seems too slow to do the symbolic calculations; need to write a routine to convert everything into numerical matrices even before multiplying them
* Need to find a way of implementing a partial trace for cvxpy

# Temporary

In [2]:
#Uttam Bhai's formula
#उत्तम भाई

for i in [True,False]:
    for j in [True,False]:
        for k in [True,False]:
            print ((i or (j and k)), ((i or j) and (i or k)))            

True True
True True
True True
True True
True True
False False
False False
False False
